FINAL PROJECT: FUNCTIONAL PROGRAMMING
E-COMMERCE SYSTEM

In [3]:
data Category = Electronics | Books | Clothing | Groceries deriving (Show, Eq)

data Product = Product {pid :: Int, pname :: String, price :: Float, category :: Category}
instance Show Product where
    show (Product id name price category) = "Product id: " ++ show id ++ ", product: " ++ show name ++ ", price: " ++ show price ++ ", category: " ++ show category

In [4]:
data LoyaltyLevel = Bronze | Silver | Gold deriving (Show, Eq)

data Customer = Customer {cid :: Int, cname :: String, loyaltyLevel :: LoyaltyLevel}

instance Show Customer where
    show (Customer id name loyaltyLevel) = "Customer id: " ++ show id ++ ", customer: " ++ show name ++ ", loyalty level: " ++ show loyaltyLevel

In [5]:
data CartItem = CartItem {product :: Product, quantity :: Int}

instance Show CartItem where
    show (CartItem product quantity) = "Item: " ++ show product ++ ", units: " ++ show quantity

In [6]:
newtype ShoppingCart = ShoppingCart [CartItem]

instance Show ShoppingCart where
    show (ShoppingCart items) = "Shopping cart:\n" ++ unlines (map (\x -> "\t" ++ show x) items)

In [7]:
newtype Stock = Stock [(Product, Int)] -- product and their available quantity

instance Show Stock where
    show (Stock items) = "Stock:\n" ++ unlines (map (\(p, i) -> "\t" ++ show p ++ ", quantity available: " ++ show i) items)

data Status = Pending | Processing | Shipped | Delivered | Cancelled deriving Show

In [8]:
data Order = Order {customer :: Customer, shoppingCart :: ShoppingCart, totalPrice :: Float, status :: Status}

instance Show Order where
    show (Order customer shoppingCart totalPrice status) = "Order for customer: " ++ show customer ++ "\n" ++ show shoppingCart 
        ++ "Total price: " ++ show totalPrice ++ ", status: " ++ show status ++ "\n"


data SearchCriterion = ById Int | ByLoyaltyLevel LoyaltyLevel | ByProductId Int | ByCategory Category | ByTotalPrice Float deriving Show

In [9]:
product1 = Product 1 "Ivan" 12 Electronics
product2 = Product 2 "Isa" 15 Clothing
product3 = Product 3 "Siro" 5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5

cart = ShoppingCart [cart1,  cart2]
cart

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

stock

customer = Customer 1 "Ivan" Gold
order = Order customer cart 17 Pending

order

Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5

Stock:
	Product id: 1, product: "Ivan", price: 12.0, category: Electronics, quantity available: 2
	Product id: 2, product: "Isa", price: 15.0, category: Clothing, quantity available: 3
	Product id: 3, product: "Siro", price: 5.0, category: Groceries, quantity available: 4

Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending

3. Price Calculation

In [10]:
calculateProductPrice :: Product -> Float
calculateProductPrice (Product _ _ p _) = p

In [11]:
calculateProductPrice product1

12.0

4. Discount Application

In [12]:
class Discountable a where
  applyDiscount :: a -> Float -> Float

instance Discountable LoyaltyLevel where
  applyDiscount Bronze p = p
  applyDiscount Silver p = p * 0.95
  applyDiscount Gold   p = p * 0.90


instance Discountable Category where
  applyDiscount Books p = p * 0.85 -- Books 15% off
  applyDiscount _     p = p


calculateOrderTotal :: Order -> Float
calculateOrderTotal (Order cust (ShoppingCart items) _ _) =
  sum [ fromIntegral (quantity it) * priceAfter (product it) | it <- items ]
  where
    priceAfter prod = applyDiscount (loyaltyLevel cust)(applyDiscount (category prod)(calculateProductPrice prod))


Testing

In [13]:
product1 = Product 1 "Ipad" 12 Electronics
product2 = Product 2 "Blouse"  15 Clothing
product3 = Product 3 "Lemons"  5 Groceries

cart1 = CartItem product1 2
cart2 = CartItem product2 5
cart  = ShoppingCart [cart1, cart2]

stock = Stock (zip [product1, product2, product3] [2, 3, 4])

customerGold   = Customer 1 "Ivan"   Gold
customerSilver = Customer 2 "Isa"  Silver
customerBronze = Customer 3 "Siro"  Bronze

orderGold   = Order customerGold   cart 0 Pending
orderSilver = Order customerSilver cart 0 Pending
orderBronze = Order customerBronze cart 0 Pending

calculateOrderTotal orderGold
calculateOrderTotal orderSilver
calculateOrderTotal orderBronze

89.1

94.05

99.0

In [14]:
productBook = Product 4 "Functional Programming for Dummies 101" 40 Books
cartBook1   = CartItem productBook 1
cartBook2   = CartItem productBook 2

cartwBook1 = ShoppingCart [cartBook1]
cartwBook2 = ShoppingCart [cartBook2]

orderBookGold1   = Order customerGold   cartwBook1 0 Pending
orderBookSilver1 = Order customerSilver cartwBook1 0 Pending
orderBookBronze1 = Order customerBronze cartwBook1 0 Pending

calculateOrderTotal orderBookGold1
calculateOrderTotal orderBookSilver1
calculateOrderTotal orderBookBronze1

30.599998

32.3

34.0

In [15]:
-- original cart + 2 books
cartMixed = ShoppingCart [cart1, cart2, cartBook2]
orderMixedGold = Order customerGold cartMixed 0 Pending

calculateOrderTotal orderMixedGold

150.29999

5. Purchase

In [16]:
instance Eq Product where
        (Product pid1 _ _ _) == (Product pid2 _ _ _) = pid1 == pid2

In [17]:
add :: CartItem -> ShoppingCart -> ShoppingCart
add item (ShoppingCart xs) = ShoppingCart (item:xs)

--add cart1 cart

In [18]:
addToCart :: CartItem -> ShoppingCart -> ShoppingCart
addToCart item (ShoppingCart []) = ShoppingCart [item]
addToCart i@(CartItem product quantity) (ShoppingCart ((CartItem p q): cartList)) 
                | product == p = ShoppingCart (CartItem p (q+quantity) : cartList)
                | otherwise = add (CartItem p q) (addToCart i (ShoppingCart cartList))

--cart

--addToCart cart1 cart

--addToCart (CartItem product3 12) cart

6. Cart Validation

In [19]:
stockQuantity :: Stock -> Product -> Int
stockQuantity (Stock xs) p
  | null matches = 0
  | otherwise    = head matches
  where
    matches = [q | (r, q) <- xs, r == p]

checkStock :: ShoppingCart -> [Product]
checkStock (ShoppingCart items) = [ p | CartItem p q <- items, stockQuantity stock p < q ]

TESTS

In [20]:
-- stock = Stock (zip [product1, product2, product3] [2, 3, 4])
cartOK = ShoppingCart [ CartItem product1 2, CartItem product2 1]
checkStock cartOK

[]

In [21]:
cartOver = ShoppingCart[ CartItem product1 2, CartItem product2 5]
checkStock cartOver

[Product id: 2, product: "Blouse", price: 15.0, category: Clothing]

In [22]:
cartMultiple = ShoppingCart[ CartItem product1 3, CartItem product2 2, CartItem product3 10]
checkStock cartMultiple

[Product id: 1, product: "Ipad", price: 12.0, category: Electronics,Product id: 3, product: "Lemons", price: 5.0, category: Groceries]

In [23]:
cartEmpty = ShoppingCart []
checkStock cartEmpty

[]

7. Order Creation

In [39]:
newtype Error = Error [Product] deriving Show

-- data Order = Order {customer :: Customer, shoppingCart :: ShoppingCart, totalPrice :: Float, status :: Status}


createOrder :: Customer -> ShoppingCart -> Either Error Order
createOrder customer c@(ShoppingCart xs) 
        | null missing = Right (Order customer c totalPrice Pending)
        | otherwise = Left (Error missing)
        where 
                missing = checkStock c
                totalPrice = calculateOrderTotal (Order customer c 0 Pending)

Test

In [34]:
createOrder customerGold cartMultiple
createOrder customerGold cartOK


Left (Error [Product id: 1, product: "Ipad", price: 12.0, category: Electronics,Product id: 3, product: "Lemons", price: 5.0, category: Groceries])

Right Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 1
Total price: 35.1, status: Pending

8. Status update

In [25]:
transitionAllowed :: Status -> Status -> Bool
transitionAllowed Pending Processing = True
transitionAllowed Pending Cancelled  = True
transitionAllowed Processing Shipped    = True
transitionAllowed Processing Cancelled  = True
transitionAllowed Shipped Delivered  = True
transitionAllowed Shipped Cancelled  = True
transitionAllowed _ _ = False

updateOrderStatus :: Order -> Status -> Maybe Order
updateOrderStatus o@(Order _ _ _ old) new
  | transitionAllowed old new = Just (o { status = new })
  | otherwise             = Nothing

Maybe is used to represent computations that might fail.It’s necessary here because an order update might or might not be allowed, so we must express both outcomes safely it will be covered eventually :)

TESTS

In [27]:
orderTest = Order customer cart 0 Pending

-- Valid transitions
updateOrderStatus orderTest Processing
updateOrderStatus orderTest Cancelled   
-- Invalid transitions
orderDelivered = Order customer cart 0 Delivered
updateOrderStatus orderDelivered Pending
updateOrderStatus orderDelivered Shipped

orderProc  = updateOrderStatus orderTest Processing
orderShip  = orderProc  >>= (`updateOrderStatus` Shipped)
orderDeliv = orderShip  >>= (`updateOrderStatus` Delivered)

orderProc
orderShip
orderDeliv

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Processing

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Cancelled

Nothing

Nothing

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Processing

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Shipped

Just Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ipad", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Blouse", price: 15.0, category: Clothing, units: 5
Total price: 0.0, status: Delivered

9. Polymorphic Search:

In [ ]:
isProductByIdInCart :: Int -> ShoppingCart -> Bool
isProductByIdInCart _ (ShoppingCart []) = False
isProductByIdInCart productId (ShoppingCart ((CartItem (Product pid _ _ _) _):xs)) 
        | productId == pid = True
        | otherwise = isProductByIdInCart productId (ShoppingCart xs)


isProductByCategoryInCart :: Category -> ShoppingCart -> Bool
isProductByCategoryInCart _ (ShoppingCart []) = False
isProductByCategoryInCart category (ShoppingCart ((CartItem (Product _ _ _ c) _):xs)) 
        | category == c = True
        | otherwise = isProductByCategoryInCart category (ShoppingCart xs)



searchOrders :: [SearchCriterion] -> [Order] -> [Order]
searchOrders [] orders = orders
searchOrders ((ById id): xs) orders = searchOrders xs (filter (\(Order (Customer cid _ _) _ _ _) -> cid == id) orders)
searchOrders ((ByLoyaltyLevel loyaltyLevel): xs) orders = searchOrders xs (filter (\(Order (Customer _ _ ll) _ _ _) -> loyaltyLevel == ll) orders)
searchOrders ((ByProductId id): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByIdInCart id cart) orders)
searchOrders ((ByCategory category): xs) orders = searchOrders xs (filter (\(Order _ cart _ _) -> isProductByCategoryInCart category cart) orders)
searchOrders ((ByTotalPrice price): xs) orders = searchOrders xs (filter (\(Order _ _ p _) -> price == p) orders)

In [128]:
product1 = Product 1 "Ivan" 12 Electronics
product2 = Product 2 "Isa" 15 Clothing
product3 = Product 3 "Siro" 5 Groceries

item1 = CartItem product1 2
item2 = CartItem product2 5
item3 = CartItem product3 10

cart1 = ShoppingCart [item1,  item2]
cart2 = ShoppingCart [item2,  item3]


customer1 = Customer 1 "Ivan" Gold
customer2 = Customer 2 "Isa" Bronze
order1 = Order customer1 cart1 17 Pending
order2 = Order customer2 cart2 20 Processing
order3 = Order customer2 cart2 17 Shipped


searchOrders [ById 2] [order1, order2, order3]
searchOrders [ByTotalPrice 17] [order1, order2, order3]
searchOrders [ByTotalPrice 17, ById 2] [order1, order2, order3]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 20.0, status: Processing
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

[Order for customer: Customer id: 1, customer: "Ivan", loyalty level: Gold
Shopping cart:
	Item: Product id: 1, product: "Ivan", price: 12.0, category: Electronics, units: 2
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
Total price: 17.0, status: Pending
,Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]

[Order for customer: Customer id: 2, customer: "Isa", loyalty level: Bronze
Shopping cart:
	Item: Product id: 2, product: "Isa", price: 15.0, category: Clothing, units: 5
	Item: Product id: 3, product: "Siro", price: 5.0, category: Groceries, units: 10
Total price: 17.0, status: Shipped
]